# 처음 설정

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/015GithubRepos/da_for_sbr')
        

import pickle
import os
import matplotlib.pyplot as plt
from collections import Counter

from preprocessing import *
from simmetric import *

experiment = 1

dataPATH = '/content/drive/MyDrive/015GithubRepos/da_for_sbr/exps'
expPATH = '/content/drive/MyDrive/015GithubRepos/da_for_sbr/exps'


# dataPATH = 'G:/내 드라이브/015GithubRepos/da_for_sbr/exps'
# expPATH = 'G:/내 드라이브/015GithubRepos/da_for_sbr/exps'


if not os.path.exists(f'exps/experiment{experiment}'):
    os.makedirs(f'exps/experiment{experiment}')


from preprocessing import *

# 불러올 데이터 선택
y_or_d = 'yoochoose'  # yoochoose or diginetica
frac = 1/64

# tra_seqs_frac 불러오기
tra_seqs_frac = load_tra_seqs_frac(experiment, y_or_d, frac)

# 아이템의 출현 횟수 구하기
allsess = []
for s in tra_seqs_frac:
  allsess += s

allaprcnt = len(allsess)
print(f'모든 세션의 아이템 출현 수 : {allaprcnt}')

# 아이템별 출현 횟수 카운트
allitemcntr = Counter(allsess)

# 아이템의 개수 구하기
nof_items = len(allitemcntr)

print(f'총 아이템 수 : {nof_items}')

loading file : exps/experiment1/yoochoose/y064_tra_seqs.pkl
124472
모든 세션의 아이템 출현 수 : 519275
총 아이템 수 : 17597


# 처음 한 번만 실행

In [ ]:
#%% 폴더 만들고 저장

if not os.path.exists(f'exps/experiment{experiment}/yoochoose'):
    os.makedirs(f'exps/experiment{experiment}/yoochoose')
    
if not os.path.exists(f'exps/experiment{experiment}/diginetica'):
    os.makedirs(f'exps/experiment{experiment}/diginetica')
    
if not os.path.exists(f'exps/experiment{experiment}/result_narm_yoochoose'):
    os.makedirs(f'exps/experiment{experiment}/result_narm_yoochoose')
    
if not os.path.exists(f'exps/experiment{experiment}/result_narm_diginetica'):
    os.makedirs(f'exps/experiment{experiment}/result_narm_diginetica')
    
if not os.path.exists(f'exps/experiment{experiment}/result_srgnn_yoochoose'):
    os.makedirs(f'exps/experiment{experiment}/result_srgnn_yoochoose')
    
if not os.path.exists(f'exps/experiment{experiment}/result_srgnn_diginetica'):
    os.makedirs(f'exps/experiment{experiment}/result_srgnn_diginetica')



## 원본데이터 불러오기, 전처리

In [ ]:
#%% 한 번만 실행하면됨

# 만들 데이터 선택
y_or_d = 'diginetica'

if y_or_d == 'yoochoose':
    filename = 'yoochoose-clicks-withHeader.dat'  # 원본 데이터 파일
    tra_sess, tes_sess, sess_clicks = prep_yoochoose(dataPATH, filename)
    fracs = [1/64, 1/128, 1/256, 1/512]
    
else:
    filename = 'train-item-views.csv'  # 원본 데이터 파일
    tra_sess, tes_sess, sess_clicks = prep_diginetica(dataPATH, filename)
    fracs = [1/1, 1/3, 1/6, 1/12]

# 원본데이터 전처리(통계량 확인차)
print(f'원본데이터 통계량')
tra_ids_ori, tra_dates_ori, tra_seqs_ori, item_dict_ori = obtian_tra(sess_clicks, tra_sess)

0it [00:00, ?it/s]

원래 데이터의 training 세션 수 : 186670
-- Starting @ %ss : 2021-12-24 15:25:09.154260+09:00
약 26초


1235380it [00:14, 87088.01it/s]


-- Reading data @ 2021-12-24 15:25:24.046016s
Splitting date 1464102000.0
원본데이터 통계량
train 세션 수 : 186670
train 아이템 수 :  43097


In [ ]:
#%% train 데이터로 저장 (마지막 아이템 떼서 라벨로)
from preprocessing import *

for f in fracs:
    split_frac = int(len(tra_sess) * f)
    tra_sess_frac = tra_sess[-split_frac:]
    
    print(f'----------------------------------------------')
    print(f'frac : 1/{int(1/f)}')
    tra_ids_frac, tra_dates_frac, tra_seqs_frac, item_dict_frac = obtian_tra(sess_clicks, tra_sess_frac)
    tes_ids, tes_dates, tes_seqs = obtian_tes(sess_clicks, tes_sess, item_dict_frac)
    save_tra_seqs_frac(experiment, y_or_d, f, tra_seqs_frac)

    # frac train데이터 저장
    tr_seqs_frac, tr_dates_frac, tr_labs_frac, tr_ids_frac = process_seqs(tra_seqs_frac, tra_dates_frac)
    te_seqs, te_dates, te_labs, te_ids = process_seqs(tes_seqs, tes_dates)
    tra_frac = (tr_seqs_frac, tr_labs_frac)
    tes = (te_seqs, te_labs)
    
    train_name = f'exps/experiment{experiment}/{y_or_d}/{y_or_d[0]}{int(1/f):03}_train.txt'
    print(f'train_name : {train_name}')
    test_name = f'exps/experiment{experiment}/{y_or_d}/{y_or_d[0]}{int(1/f):03}_test.txt'
    print(f'test_name : {test_name}')
    pickle.dump(tra_frac, open(train_name, 'wb'))
    pickle.dump(tes, open(test_name, 'wb'))

----------------------------------------------
frac : 1/1
train 세션 수 : 186670
train 아이템 수 :  43097
test 세션 총 개수 : 15963
test 데이터 아이템 수 :  21131
tra_seqs save dir : exps/experiment1/diginetica/d001_tra_seqs.pkl
train_name : exps/experiment1/diginetica/d001_train.txt
test_name : exps/experiment1/diginetica/d001_test.txt
----------------------------------------------
frac : 1/3
train 세션 수 : 62223
train 아이템 수 :  36427
test 세션 총 개수 : 15644
test 데이터 아이템 수 :  19651
tra_seqs save dir : exps/experiment1/diginetica/d003_tra_seqs.pkl
train_name : exps/experiment1/diginetica/d003_train.txt
test_name : exps/experiment1/diginetica/d003_test.txt
----------------------------------------------
frac : 1/6
train 세션 수 : 31111
train 아이템 수 :  29129
test 세션 총 개수 : 15000
test 데이터 아이템 수 :  17063
tra_seqs save dir : exps/experiment1/diginetica/d006_tra_seqs.pkl
train_name : exps/experiment1/diginetica/d006_train.txt
test_name : exps/experiment1/diginetica/d006_test.txt
------------------------------------------

In [ ]:
from simmetric import *

co_mat_win = get_co_matrix_win(tra_seqs_frac, 5, True, True)
save_mat_as_csr(co_mat_win, 'coo_mat', experiment, y_or_d, frac)

pmi_mat = ppmi(co_mat_win, True)
save_mat_as_csr(pmi_mat, 'pmi_mat', experiment, y_or_d, frac)

jac_mat = get_jaccard_mat(tra_seqs_frac, 5, diag_freq = True, remove_dup = True)
save_mat_as_csr(jac_mat, 'jac_mat', experiment, y_or_d, frac)
,v
tan_mat = get_tanimoto(tra_seqs_frac, window_size=5, diag_freq=True, remove_dup=True)
save_mat_as_csr(tan_mat, 'tan_mat', experiment, y_or_d, frac)

cos_mat_win = get_cosine(co_mat_win)
save_mat_as_csr(cos_mat_win, 'cos_mat', experiment, y_or_d, frac)

w2v_mat = get_w2v_model(tra_seqs_frac, nof_items)
save_mat_as_csr(w2v_mat, 'w2v_mat', experiment, y_or_d, frac)

저장파일 : exps/experiment1/yoochoose/y064_csr_coo_mat.pkl


/content/drive/MyDrive/015GithubRepos/da_for_sbr/simmetric.py:104: RuntimeWarning: invalid value encountered in long_scalars
  pmi = np.log2(C[i, j] * N / (S[j]*S[i]) + eps)  # np.log2(0)이 음의 무한대가 되는 것을 막기위해 eps 사용함


0.0 % 완료
17.047391749062392 % 완료
34.094783498124784 % 완료
51.14217524718718 % 완료
68.18956699624957 % 완료
85.23695874531197 % 완료
저장파일 : exps/experiment1/yoochoose/y064_csr_pmi_mat.pkl
0.0 % 완료
5.682463916354131 % 완료
11.364927832708261 % 완료
17.047391749062392 % 완료
22.729855665416522 % 완료
28.412319581770657 % 완료
34.094783498124784 % 완료
39.777247414478914 % 완료
45.459711330833045 % 완료
51.14217524718718 % 완료
56.82463916354131 % 완료
62.507103079895444 % 완료
68.18956699624957 % 완료
73.8720309126037 % 완료
79.55449482895783 % 완료
85.23695874531197 % 완료
90.91942266166609 % 완료
96.60188657802023 % 완료
저장파일 : exps/experiment1/yoochoose/y064_csr_jac_mat.pkl
10.00% 완료
20.00% 완료
30.00% 완료
40.00% 완료
50.00% 완료
60.00% 완료
70.00% 완료
80.00% 완료
90.00% 완료
100.00% 완료
저장파일 : exps/experiment1/yoochoose/y064_csr_tan_mat.pkl
10.00% 완료
20.00% 완료
30.00% 완료
40.00% 완료
50.00% 완료
60.00% 완료
70.00% 완료
80.00% 완료
90.00% 완료
저장파일 : exps/experiment1/yoochoose/y064_csr_cos_mat.pkl


In [ ]:
w2v_mat = get_w2v_model(tra_seqs_frac, nof_items)
save_mat_as_csr(w2v_mat, 'w2v_mat', experiment, y_or_d, frac)

저장파일 : exps/experiment1/yoochoose/y064_csr_w2v_mat.pkl


# 데이터 증강

In [3]:
#%% falog2만들기, hs딕셔너리(msd, most hightest similarity dictionary) 불러오기

import math
falog2 = {k:math.log2(v+1) for k, v in allitemcntr.items()}

from simmetric import *
msd = load_msd('coo', experiment, y_or_d, frac)

nof_sessions = len(tra_seqs_frac)


In [4]:
#%% hyperparameter setting

alpha = 500
mu1 = 0
sigma1 = 3

beta = 500
mu2 = 1
sigma2 = 0.4

#%% 출현빈도 그리기 ##############################################

min(falog2_d.values())
min(allitemcntr.values())

mu1, sigma1 = M, 3

# x, y 생성
x = [falog2_d[i] for i in range(1, nof_items+1)]
y = [alpha * gaussian(mu1, sigma1, _) for _ in x]
plt.scatter(x, y)

# 가우시안 선
x = np.linspace(-3, 15, 1000)
y = gaussian(mu1, sigma1, x) * alpha
plt.plot(x, y)

# 수선
plt.axvline(x=mu1, color='r')
plt.axvline(x=M, color='r')


plt.xlabel('log2(FA)', fontsize=20)
plt.title(f'100 X N(mu1={mu1:.2f}, sigma1={sigma1})', fontsize=20)
#####################################################################



theta = [alpha, mu1, sigma1, beta, mu2, sigma2]

def cal_score(logfa: float , hsvalue: float, theta) -> float:

  alpha, mu1, sigma1, beta, mu2, sigma2 = theta[0], theta[1], theta[2], theta[3], theta[4], theta[5]

  alphaterm = alpha * gaussian(mu1, sigma1, logfa)
  betaterm = beta * gaussian(mu2, sigma2, hsvalue)

  score = alphaterm + betaterm

  return score

score = {i : cal_score(falog2[i], msd[i][1], theta) for i in range(1, nof_items+1)}

In [5]:
sess_score_mean = dict()
for sid, s in enumerate(tra_seqs_frac):
  scorelist = [score[item] for idx, item in enumerate(s)]
  sess_score_mean[sid] = (np.mean(scorelist), len(s))  # sess_score_mean[session id] = (session's score mean, session length)

 
aug_target = sorted(sess_score_mean.items(), key=lambda x: (x[1][0], -x[1][1]), reverse=True)
aug_target[:10]

[(156, (556.5071653814214, 2)),
 (223, (556.5071653814214, 2)),
 (1219, (556.5071653814214, 2)),
 (1303, (556.5071653814214, 2)),
 (2170, (556.5071653814214, 2)),
 (2501, (556.5071653814214, 2)),
 (2731, (556.5071653814214, 2)),
 (2846, (556.5071653814214, 2)),
 (3113, (556.5071653814214, 2)),
 (3221, (556.5071653814214, 2))]

In [6]:
aug_target = [_[0] for _ in aug_target]
aug_target[:10]

[156, 223, 1219, 1303, 2170, 2501, 2731, 2846, 3113, 3221]

### 증강 대상 세션 정렬했다(이제 증강 안되는 세션은 건너뛰기)

In [7]:
ar = 0.5  # augmentation rate
nof_aug = int(nof_sessions * ar)  # number of augmentation
print(f'number of sessions : {nof_sessions}')
print(f'augmentation rate : {ar}')
print(f'number of augmentation : {nof_aug}')


cr = 0.5  # choice rate
print(f'choice rate : {ar}')

number of sessions : 124472
augmentation rate : 0.5
number of augmentation : 62236
choice rate : 0.5


In [8]:
cnt = 0  # 증강 성공할때마다 하나씩 카운트
aug_sess = []  # 증강 완료된 세션을 모음


for rank, sid in enumerate(aug_target):

  s = tra_seqs_frac[sid].copy()  # 안변하고 참고하는거
  sess = tra_seqs_frac[sid].copy()  # 실제로 변하는거
  slen = len(sess)

  scorelist = np.array([score[item] for item in sess])

  np.random.choice(range(len(sess)), math.floor(len(sess) * cr))

  p = np.array(scorelist) / np.sum(scorelist)

  augidxs = np.random.choice(range(len(sess)), math.floor(len(sess) * cr), replace=False, p=p)

  nof_sub = np.random.choice(slen + 1, 1)[0]  # 대체할 개수
  sub_it_idxs = augidxs[:nof_sub]
  isrt_it_idxs = augidxs[nof_sub:]


  # sforsub에 대체
  for sitidx in sub_it_idxs:
    # 가장 유사한 아이템 찾기 -> 자기 자신과만 유사한 아이템만 아니면 대체
    mostsimitem_s = msd[sess[sitidx]][0]
    if sess[sitidx] != mostsimitem_s:
      sess[sitidx] = mostsimitem_s  # 대체
  
  # 삽입할 아이템 구하기
  isrt_items = []  # 삽입할 아이템을 모음
  isrt_it_idxs_over = []  # 삽입할 인덱스를 모음
  for iitidx in isrt_it_idxs:
    # 가장 유사한 아이템 찾기 -> 자기자신과만 유사한 아이템 아니면 삽입
    mostsimitem_i = msd[sess[iitidx]][0]
    if sess[iitidx] != mostsimitem_i:
      isrt_items.append(mostsimitem_i)  # 삽입할 아이템
      isrt_it_idxs_over.append(iitidx)  # 삽입할 인덱스

  sess = np.insert(np.asarray(sess), isrt_it_idxs_over, isrt_items)  # 삽입
  sess = list(sess)

  if s != sess:
    cnt += 1
    print(f'************성공**************')
    aug_sess.append(sess)

  if cnt == nof_aug:
    break
  
  print(f'순위 : {rank}')
  print(f'원래 세션   : {s}')
  print(f'증강된 세션 : {sess}')
  print(f'p : {p}')
  print(f'score mean: {np.mean(scorelist):.2f}')
  print()


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
원래 세션   : [1016, 1992, 1016]
증강된 세션 : [1016, 1016, 1016]
p : [0.30481964 0.39036071 0.30481964]
score mean: 42.07

************성공**************
순위 : 62377
원래 세션   : [284, 163]
증강된 세션 : [136, 284, 163]
p : [0.45929548 0.54070452]
score mean: 42.07

************성공**************
순위 : 62378
원래 세션   : [1340, 942]
증강된 세션 : [1340, 82, 942]
p : [0.54911718 0.45088282]
score mean: 42.07

************성공**************
순위 : 62379
원래 세션   : [18, 18, 340]
증강된 세션 : [18, 5, 340]
p : [0.38267312 0.38267312 0.23465375]
score mean: 42.07

************성공**************
순위 : 62380
원래 세션   : [3584, 1239, 564, 174, 27, 1000]
증강된 세션 : [174, 27, 564, 27, 27, 1000]
p : [0.1542418  0.14495428 0.19355868 0.20529056 0.19886831 0.10308638]
score mean: 42.07

************성공**************
순위 : 62381
원래 세션   : [346, 368, 6133, 368, 2710, 6133]
증강된 세션 : [904, 368, 6133, 260, 2710, 578]
p : [0.16819539 0.15094507 0.1731661  0.15094507 0.18358228 0.1731661 ]
score mean: 42.07

*********

In [9]:
len(aug_sess)

62236

In [10]:
def crop_seqs(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        for i in range(1, len(seq)):
            tar = seq[-i]  # tar : 맨 뒤 아이템 하나씩 잘라 라벨로
            labs += [tar]
            out_seqs += [seq[:-i]]
    return out_seqs, labs

In [11]:
aug_tra, aug_tra_l = crop_seqs(tra_seqs_frac + aug_sess)
aug_tra_final = (aug_tra, aug_tra_l)

In [12]:
filename_save = f'exps/experiment{experiment}/{y_or_d}/{y_or_d[0]}{int(1/frac):03}_testhaha.txt'
print(filename_save)
pickle.dump(aug_tra_final, open(filename_save, 'wb'))

exps/experiment1/yoochoose/y064_testhaha.txt


# 모델 돌리기

In [13]:
from models import *

In [14]:
trainfiles = [
              'y064_train.txt', 'y064_train2.txt', 'y064_train3.txt', 'y064_train4.txt', 'y064_train5.txt', 'y064_testhaha.txt'
              ]
            
narm(trainfiles, 1, 'yoochoose', 1/64)

Namespace(batch_size=512, embed_dim=50, epoch=100, hidden_size=100, lr=0.001, lr_dc=0.1, lr_dc_step=80, test=False, test_file='exps/', topk=20, train_file='exps/', valid=False, valid_portion=0.1)
--------------------------------------------------
Dataset info:
Number of sessions: 394803
--------------------------------------------------
--------------------------------------------------
Dataset info:
Number of sessions: 55424
--------------------------------------------------
아이템 개수 : 17597
1 ~ 17597
모델에 들어가는 수 : 17598


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://git

Epoch 0 : Recall@20: 33.9736, MRR@20: 16.5600 	y064_train.txt 	 1/6 	 y064_test.txt
Epoch 1 : Recall@20: 44.2947, MRR@20: 20.0250 	y064_train.txt 	 1/6 	 y064_test.txt
Epoch 2 : Recall@20: 49.3478, MRR@20: 21.5470 	y064_train.txt 	 1/6 	 y064_test.txt
Epoch 3 : Recall@20: 52.6770, MRR@20: 22.9744 	y064_train.txt 	 1/6 	 y064_test.txt
Epoch 4 : Recall@20: 55.5440, MRR@20: 24.3007 	y064_train.txt 	 1/6 	 y064_test.txt
Epoch 5 : Recall@20: 57.6584, MRR@20: 24.9874 	y064_train.txt 	 1/6 	 y064_test.txt
Epoch 6 : Recall@20: 59.1080, MRR@20: 25.5067 	y064_train.txt 	 1/6 	 y064_test.txt
Epoch 7 : Recall@20: 60.4358, MRR@20: 25.9430 	y064_train.txt 	 1/6 	 y064_test.txt
Epoch 8 : Recall@20: 61.7635, MRR@20: 26.4286 	y064_train.txt 	 1/6 	 y064_test.txt
Epoch 9 : Recall@20: 62.5197, MRR@20: 26.7732 	y064_train.txt 	 1/6 	 y064_test.txt
Epoch 10 : Recall@20: 63.5088, MRR@20: 27.0424 	y064_train.txt 	 1/6 	 y064_test.txt
Epoch 11 : Recall@20: 64.0804, MRR@20: 27.4615 	y064_train.txt 	 1/6 	 y064

([33.97362385321101,
  44.294724770642205,
  49.34776376146789,
  52.67703555045872,
  55.54400802752294,
  57.658400229357795,
  59.10801318807339,
  60.43577981651376,
  61.76354644495413,
  62.51971043577982,
  63.50881594036697,
  64.08041857798165,
  64.63410263761467,
  65.27379587155964,
  65.48881880733946,
  66.14284690366972,
  66.42416857798165,
  66.50121846330275,
  66.9151376146789,
  67.23229644495413,
  67.46702981651376,
  67.59783543577981,
  67.90603497706422,
  68.01354644495413,
  68.14614392201835,
  68.27336582568807,
  68.54572821100918,
  68.60485951834863,
  68.72491399082568,
  68.69086869266054,
  68.92918577981652,
  69.17287844036697,
  69.24096903669725,
  69.2947247706422,
  69.37894208715596,
  69.45599197247707,
  69.55096043577981,
  69.54379300458714,
  69.64234518348624,
  69.61367545871559,
  69.7265625,
  69.5724627293578,
  69.77852637614679,
  69.78211009174312,
  69.94158543577981,
  69.88782970183486,
  70.02042717889908,
  69.85736811926606,
